In [1]:
import pandas as pd

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import re
import os
import nltk
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download("punkt")
nltk.download('wordnet')
import math
import operator

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [212]:
def punctuation(txt):
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~=+'''
  for char in txt:
    if char in punc:
      txt = txt.replace(char, " ")
  return txt

def numbering(txt):
  for char in txt:
    if char in "0":
      txt = txt.replace(char, " zero ")
    elif char in "1":
      txt = txt.replace(char, " one ")
    elif char in "2":
      txt = txt.replace(char, " two ")
    elif char in "3":
      txt = txt.replace(char, " three ")
    elif char in "4":
      txt = txt.replace(char, " four ")
    elif char in "5":
      txt = txt.replace(char, " five ")
    elif char in "6":
      txt = txt.replace(char, " six ")
    elif char in "7":
      txt = txt.replace(char, " seven ")
    elif char in "8":
      txt = txt.replace(char, " eight ")
    elif char in "9":
      txt = txt.replace(char, " nine ")
  return txt

def lemme(txt):
  txt=nltk.word_tokenize(txt)
  temp=[]
  for i in txt:
    temp.append(WordNetLemmatizer().lemmatize(i))
  return temp

def stemming(txt):
  txt=nltk.word_tokenize(txt)
  temp=[]
  for i in txt:
    temp.append(PorterStemmer().stem(i))
  return temp

def lessThan2(txt):
  temp=[]
  for i in txt:
    if(len(i)>1):
      temp.append(i)
  return temp

def accuracy(y_true,y_pred):
  count=0
  for i in range(0,len(y_true)):
    if y_true[i]==y_pred[i]:
      count=count+1
  return (count/len(y_true))

def confusion_mat(y_true,y_pred):
  dic={"talk.politics.misc":0,"sci.med":1,"sci.space":2,"comp.graphics":3,"rec.sport.hockey":4}
  y_true_change=[]
  y_pred_change=[]
  for i in range(0,len(y_true)):
    y_true_change.append(dic[y_true[i]])
    y_pred_change.append(dic[y_pred[i]])
  n = len(np.unique(y_true)) 
  mat = np.zeros((n, n))
  for i in range(len(y_true)):
    mat[y_true_change[i]][y_pred_change[i]] += 1
  return mat

In [117]:
def process(txt):

  #punctuation removal and number conversion
  txt=punctuation(txt)
  
  
    
  # #lower the text
  # txt_lower=txt.lower()
  # print(txt_lower)

  #remove stopwords
  txt=remove_stopwords(txt)
  
  

  #convert number
  txt=numbering(txt)
  

  #lower the text
  txt_lower=txt.lower()
  

  #tokenization & lemmetization
  #txt_tokenize=lemme(txt_lower)
  

  #stemmed
  txt_stemmed=stemming(txt_lower)
  

  #remove word, size<2
  txt_length2=lessThan2(txt_stemmed)
  

  #remove duplicates 
  #txt_tokenize =set(txt_tokenize)
  
  return txt_length2 

In [160]:
def process(txt):

  #punctuation removal
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~=+'''
  for ele in txt:
    if ele in punc:
        txt = txt.replace(ele, " ")
  
  #lower the text
  txt_lower=txt.lower()

  #remove stopwords
  txt_stopwords=remove_stopwords(txt_lower)
  
  #tokenization
  txt_tokenize=nltk.word_tokenize(txt_stopwords)

  #remove blank spaces
  for word in txt_tokenize:
    word=word.strip()  

  #remove duplicates 
  #txt_tokenize =set(txt_tokenize)
  
  return txt_tokenize 

In [161]:
my_list1=os.listdir("/content/drive/MyDrive/News");

In [162]:
my_list1

['sci.med',
 'rec.sport.hockey',
 'talk.politics.misc',
 'comp.graphics',
 'sci.space']

make dataframe

In [163]:
dataframe=pd.DataFrame(columns=["content","label"])

In [164]:
listOfContent=[]
listOfLabel=[]
for i in my_list1:
  folder=os.listdir("/content/drive/MyDrive/News/"+i)
  for j in folder:
    file=open("/content/drive/MyDrive/News/"+i+"/"+j,"r",errors="ignore",encoding ="utf-8")
    my_txt=file.read()
    processed_txt=process(my_txt)
    listOfContent.append(processed_txt)
    listOfLabel.append(i)

In [165]:
dataframe["content"]=listOfContent
dataframe["label"]=listOfLabel

In [166]:
dataframe

,content,label
0,"[path, cantaloupe, srv, cs, cmu, edu, das, new...",sci.med
1,"[xref, cantaloupe, srv, cs, cmu, edu, rec, nud...",sci.med
2,"[xref, cantaloupe, srv, cs, cmu, edu, misc, ki...",sci.med
3,"[path, cantaloupe, srv, cs, cmu, edu, crabappl...",sci.med
4,"[path, cantaloupe, srv, cs, cmu, edu, crabappl...",sci.med
...,...,...
4995,"[path, cantaloupe, srv, cs, cmu, edu, crabappl...",sci.space
4996,"[path, cantaloupe, srv, cs, cmu, edu, crabappl...",sci.space
4997,"[newsgroups, sci, space, path, cantaloupe, srv...",sci.space
4998,"[xref, cantaloupe, srv, cs, cmu, edu, sci, spa...",sci.space


In [374]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframe, dataframe["label"], test_size=0.20, random_state=42,stratify=dataframe["label"])

In [375]:
#creating dictionary to store {terms:{class:count}}
main_dict2={}
unique_words2=set()
for i in range(0,len(X_train)):
  data=X_train.iloc[i]
  temp_list=data["content"]
  label=data["label"]
  unique_words2.update(temp_list)
  for token in temp_list:
      if token in main_dict2:
        if label in main_dict2[token]:
          main_dict2[token][label]=main_dict2[token][label]+1
        else:
          main_dict2[token][label]=1
      else:
        main_dict2[token]={}
        main_dict2[token][label]=1

#making bag of words
word_list=[]
for i in unique_words2:
  word_list.append(i)

#making a new dataframe to store tficf value
matrix=pd.DataFrame()
matrix["terms"]=word_list
for i in my_list1:
  matrix[i]=0
matrix.index = list(matrix["terms"])

In [376]:
#filling  Tficf value in dataframe
for i in word_list:
  class_list=main_dict2[i]
  icf_value=math.log((5/len(class_list)),2)
  for j in my_list1:
    if j in main_dict2[i]:
      matrix.loc[i,j]=main_dict2[i][j]*icf_value

In [377]:
#function for selecting k best features
def getKBestFeatures(k):
  listing=set()
  for i in my_list1:
    value=matrix[i]
    key=matrix["terms"]
    term_value_dict = dict(zip(key,value))
    sorted_dict = sorted(term_value_dict.items(), key=operator.itemgetter(1),reverse=True)
    i=0
    while i<k:
      listing.add(sorted_dict[i][0])
      i=i+1
  return listing

#calling to select best features
best_features=getKBestFeatures(13)

calculating count of the documnets in which that word present for each class

In [378]:
dict_term_y={}
dict_term_n={}
for i in best_features:
  for j in my_list1:
    df=X_train[X_train["label"]==j]
    for k in range(0,len(df)):
      if i in df.iloc[k]["content"]:
        if i in dict_term_y:
          if j in dict_term_y[i]:
            dict_term_y[i][j]=dict_term_y[i][j]+1
          else:
            dict_term_y[i][j]=1
        else:
          dict_term_y[i]={}
          dict_term_y[i][j]=1
      else:
        if i in dict_term_n:
          if j in dict_term_n[i]:
            dict_term_n[i][j]=dict_term_n[i][j]+1
          else:
            dict_term_n[i][j]=1
        else:
          dict_term_n[i]={}
          dict_term_n[i][j]=1

class-count dictionary

In [379]:
class_totalCount={"talk.politics.misc":800,"sci.med":800,"sci.space":800,"comp.graphics":800,"rec.sport.hockey":800}

storing probabilities for important features

In [380]:
prob_dict_y={}
prob_dict_n={}
for i in best_features:
  prob_dict_y[i]=[]
  prob_dict_n[i]=[]
  for j in my_list1:
    if j in dict_term_y[i]:
      count=dict_term_y[i][j]
    else:
      count=0
    prob_dict_y[i].append((count)/(class_totalCount[j]))
    if j in dict_term_n[i]:
      count=dict_term_n[i][j]
    else:
      count=0
    prob_dict_n[i].append((count)/(class_totalCount[j]))

model training

In [381]:
y_pred=[]
for i in range(0,len(X_test)):
  data=X_test.iloc[i]
  temp_list=data["content"]
  k=('sci.med','rec.sport.hockey','talk.politics.misc','comp.graphics','sci.space')
  v=(1,1,1,1,1)
  d=dict(zip(k,v))
  for j in best_features:
    if j in temp_list:
      d['sci.med']=d['sci.med']*prob_dict_y[j][0]
      d['rec.sport.hockey']=d['rec.sport.hockey']*prob_dict_y[j][1]
      d['talk.politics.misc']=d['talk.politics.misc']*prob_dict_y[j][2]
      d['comp.graphics']=d['comp.graphics']*prob_dict_y[j][3]
      d['sci.space']=d['sci.space']*prob_dict_y[j][4]
    else:
      d['sci.med']=d['sci.med']*prob_dict_n[j][0]
      d['rec.sport.hockey']=d['rec.sport.hockey']*prob_dict_n[j][1]
      d['talk.politics.misc']=d['talk.politics.misc']*prob_dict_n[j][2]
      d['comp.graphics']=d['comp.graphics']*prob_dict_n[j][3]
      d['sci.space']=d['sci.space']*prob_dict_n[j][4]
  sorted_dict = sorted(d.items(), key=operator.itemgetter(1),reverse=True) 
  y_pred.append(sorted_dict[0][0])

making prediction

In [382]:
print(accuracy(list(y_test),y_pred))
print(confusion_mat(list(y_test),y_pred))

0.911
[[191.   1.   8.   0.   0.]
 [  1. 193.   1.   5.   0.]
 [  0.  68. 131.   1.   0.]
 [  0.   3.   1. 196.   0.]
 [  0.   0.   0.   0. 200.]]


# for 70:30 splitting

In [383]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframe, dataframe["label"], test_size=0.30, random_state=42,stratify=dataframe["label"])

In [384]:
#creating dictionary to store {terms:{class:count}}
main_dict2={}
unique_words2=set()
for i in range(0,len(X_train)):
  data=X_train.iloc[i]
  temp_list=data["content"]
  label=data["label"]
  unique_words2.update(temp_list)
  for token in temp_list:
      if token in main_dict2:
        if label in main_dict2[token]:
          main_dict2[token][label]=main_dict2[token][label]+1
        else:
          main_dict2[token][label]=1
      else:
        main_dict2[token]={}
        main_dict2[token][label]=1

#making bag of words
word_list=[]
for i in unique_words2:
  word_list.append(i)

#making a new dataframe to store tficf value
matrix=pd.DataFrame()
matrix["terms"]=word_list
for i in my_list1:
  matrix[i]=0
matrix.index = list(matrix["terms"])

In [385]:
#filling  Tficf value in dataframe
for i in word_list:
  class_list=main_dict2[i]
  icf_value=math.log((5/len(class_list)),2)
  for j in my_list1:
    if j in main_dict2[i]:
      matrix.loc[i,j]=main_dict2[i][j]*icf_value

In [386]:
#function for selecting k best features
def getKBestFeatures(k):
  listing=set()
  for i in my_list1:
    value=matrix[i]
    key=matrix["terms"]
    term_value_dict = dict(zip(key,value))
    sorted_dict = sorted(term_value_dict.items(), key=operator.itemgetter(1),reverse=True)
    i=0
    while i<k:
      listing.add(sorted_dict[i][0])
      i=i+1
  return listing

#calling to select best features
best_features=getKBestFeatures(13)

calculating count of the documnets in which that word present for each class

In [387]:
dict_term_y={}
dict_term_n={}
for i in best_features:
  for j in my_list1:
    df=X_train[X_train["label"]==j]
    for k in range(0,len(df)):
      if i in df.iloc[k]["content"]:
        if i in dict_term_y:
          if j in dict_term_y[i]:
            dict_term_y[i][j]=dict_term_y[i][j]+1
          else:
            dict_term_y[i][j]=1
        else:
          dict_term_y[i]={}
          dict_term_y[i][j]=1
      else:
        if i in dict_term_n:
          if j in dict_term_n[i]:
            dict_term_n[i][j]=dict_term_n[i][j]+1
          else:
            dict_term_n[i][j]=1
        else:
          dict_term_n[i]={}
          dict_term_n[i][j]=1

class-count dictionary

In [388]:
class_totalCount={"talk.politics.misc":700,"sci.med":700,"sci.space":700,"comp.graphics":700,"rec.sport.hockey":700}

storing probabilities for important features

In [389]:
prob_dict_y={}
prob_dict_n={}
for i in best_features:
  prob_dict_y[i]=[]
  prob_dict_n[i]=[]
  for j in my_list1:
    if j in dict_term_y[i]:
      count=dict_term_y[i][j]
    else:
      count=0
    prob_dict_y[i].append((count)/(class_totalCount[j]))
    if j in dict_term_n[i]:
      count=dict_term_n[i][j]
    else:
      count=0
    prob_dict_n[i].append((count)/(class_totalCount[j]))

model training

In [390]:
y_pred=[]
for i in range(0,len(X_test)):
  data=X_test.iloc[i]
  temp_list=data["content"]
  k=('sci.med','rec.sport.hockey','talk.politics.misc','comp.graphics','sci.space')
  v=(1,1,1,1,1)
  d=dict(zip(k,v))
  for j in best_features:
    if j in temp_list:
      d['sci.med']=d['sci.med']*prob_dict_y[j][0]
      d['rec.sport.hockey']=d['rec.sport.hockey']*prob_dict_y[j][1]
      d['talk.politics.misc']=d['talk.politics.misc']*prob_dict_y[j][2]
      d['comp.graphics']=d['comp.graphics']*prob_dict_y[j][3]
      d['sci.space']=d['sci.space']*prob_dict_y[j][4]
    else:
      d['sci.med']=d['sci.med']*prob_dict_n[j][0]
      d['rec.sport.hockey']=d['rec.sport.hockey']*prob_dict_n[j][1]
      d['talk.politics.misc']=d['talk.politics.misc']*prob_dict_n[j][2]
      d['comp.graphics']=d['comp.graphics']*prob_dict_n[j][3]
      d['sci.space']=d['sci.space']*prob_dict_n[j][4]
  sorted_dict = sorted(d.items(), key=operator.itemgetter(1),reverse=True) 
  y_pred.append(sorted_dict[0][0])

making prediction

In [391]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

0.918
[[295   0   7   0   0]
 [  0 299   0   0   0]
 [  4   1 289  96   3]
 [  1   0   2 198   1]
 [  0   0   2   6 296]]


# for 50:50 split

In [392]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataframe, dataframe["label"], test_size=0.50, random_state=42,stratify=dataframe["label"])

In [393]:
#creating dictionary to store {terms:{class:count}}
main_dict2={}
unique_words2=set()
for i in range(0,len(X_train)):
  data=X_train.iloc[i]
  temp_list=data["content"]
  label=data["label"]
  unique_words2.update(temp_list)
  for token in temp_list:
      if token in main_dict2:
        if label in main_dict2[token]:
          main_dict2[token][label]=main_dict2[token][label]+1
        else:
          main_dict2[token][label]=1
      else:
        main_dict2[token]={}
        main_dict2[token][label]=1

#making bag of words
word_list=[]
for i in unique_words2:
  word_list.append(i)

#making a new dataframe to store tficf value
matrix=pd.DataFrame()
matrix["terms"]=word_list
for i in my_list1:
  matrix[i]=0
matrix.index = list(matrix["terms"])

In [394]:
#filling  Tficf value in dataframe
for i in word_list:
  class_list=main_dict2[i]
  icf_value=math.log((5/len(class_list)),2)
  for j in my_list1:
    if j in main_dict2[i]:
      matrix.loc[i,j]=main_dict2[i][j]*icf_value

In [395]:
#function for selecting k best features
def getKBestFeatures(k):
  listing=set()
  for i in my_list1:
    value=matrix[i]
    key=matrix["terms"]
    term_value_dict = dict(zip(key,value))
    sorted_dict = sorted(term_value_dict.items(), key=operator.itemgetter(1),reverse=True)
    i=0
    while i<k:
      listing.add(sorted_dict[i][0])
      i=i+1
  return listing

#calling to select best features
best_features=getKBestFeatures(15)

calculating count of the documnets in which that word present for each class

In [396]:
dict_term_y={}
dict_term_n={}
for i in best_features:
  for j in my_list1:
    df=X_train[X_train["label"]==j]
    for k in range(0,len(df)):
      if i in df.iloc[k]["content"]:
        if i in dict_term_y:
          if j in dict_term_y[i]:
            dict_term_y[i][j]=dict_term_y[i][j]+1
          else:
            dict_term_y[i][j]=1
        else:
          dict_term_y[i]={}
          dict_term_y[i][j]=1
      else:
        if i in dict_term_n:
          if j in dict_term_n[i]:
            dict_term_n[i][j]=dict_term_n[i][j]+1
          else:
            dict_term_n[i][j]=1
        else:
          dict_term_n[i]={}
          dict_term_n[i][j]=1

class-count dictionary

In [397]:
class_totalCount={"talk.politics.misc":500,"sci.med":500,"sci.space":500,"comp.graphics":500,"rec.sport.hockey":500}

storing probabilities for important features

In [398]:
prob_dict_y={}
prob_dict_n={}
for i in best_features:
  prob_dict_y[i]=[]
  prob_dict_n[i]=[]
  for j in my_list1:
    if j in dict_term_y[i]:
      count=dict_term_y[i][j]
    else:
      count=0
    prob_dict_y[i].append((count)/(class_totalCount[j]))
    if j in dict_term_n[i]:
      count=dict_term_n[i][j]
    else:
      count=0
    prob_dict_n[i].append((count)/(class_totalCount[j]))

model training

In [399]:
y_pred=[]
for i in range(0,len(X_test)):
  data=X_test.iloc[i]
  temp_list=data["content"]
  k=('sci.med','rec.sport.hockey','talk.politics.misc','comp.graphics','sci.space')
  v=(1,1,1,1,1)
  d=dict(zip(k,v))
  for j in best_features:
    if j in temp_list:
      d['sci.med']=d['sci.med']*prob_dict_y[j][0]
      d['rec.sport.hockey']=d['rec.sport.hockey']*prob_dict_y[j][1]
      d['talk.politics.misc']=d['talk.politics.misc']*prob_dict_y[j][2]
      d['comp.graphics']=d['comp.graphics']*prob_dict_y[j][3]
      d['sci.space']=d['sci.space']*prob_dict_y[j][4]
    else:
      d['sci.med']=d['sci.med']*prob_dict_n[j][0]
      d['rec.sport.hockey']=d['rec.sport.hockey']*prob_dict_n[j][1]
      d['talk.politics.misc']=d['talk.politics.misc']*prob_dict_n[j][2]
      d['comp.graphics']=d['comp.graphics']*prob_dict_n[j][3]
      d['sci.space']=d['sci.space']*prob_dict_n[j][4]
  sorted_dict = sorted(d.items(), key=operator.itemgetter(1),reverse=True) 
  y_pred.append(sorted_dict[0][0])

making prediction

In [400]:
from sklearn.metrics import accuracy_score,confusion_matrix
print(accuracy_score(y_pred,y_test))
print(confusion_matrix(y_pred,y_test))

0.9196
[[493   0   9   0   0]
 [  0 490   0   0   0]
 [  6  10 480 134  13]
 [  1   0   9 350   1]
 [  0   0   2  16 486]]
